# Introdução à Inteligência Artificial (2021/2022)

## Problemas de Satisfação de Restrições - Constraint Satisfaction Problems (CSP)

<img src="Imagens/sudoku.gif" width=300 height=300 />

### Introdução

Nesta aula vamos formular e resolver problemas de satisfação de restrições - **Constraint Satisfaction Problems (CSP)**.
O contexto é o analisado nas aulas T e TP em que um CSP é caracterizado por 3 componentes:
- **Variáveis** - para as quais se quer encontrar uma afectação (*assigment*) completa e consistente (solução); podem ser discretas ou contínuas.
- **Domínios** - indicam os valores possíveis das variáveis; podem ser finitos ou infinitos.
- **Restrições** - especificam combinações possíveis de valores para subconjuntos de variáveis; podem ser 1) unárias por exemplo <(X1): X1=5)>, 2) binárias, por exemplo `<(X1,X2): X1=X2)>`, ou de 3) ordem superior (por exemplo `<(X1,X2,X3), alldiff(X1,X2,X3>`.

Neste contexto, **em CSP**:

* **Estado** - é um conjunto de variaveis com valores nos seus domínios (afectação/assignment); uma afectação pode ser 1) global, se todas as variáveis tem valor, ou parcial, caso contrário; e 2) consistente, se  satisfaz todas as restrições, ou inconsistente, caso contrário. 
* **Inferência e Procura** - é efectuada usando propagação de restrições e, quando ainda há diversas soluções possíveis, algoritmos de procura que usam heurísticas genéricas em vez de heurística específica do problema, como acontecia na resolução dos Problemas de Espeços de Estados (PEE).
* **Solução** - é uma afectação completa e consistente.

**A resolução de um CSP tem 2 passos**:

**Passo 1)** Formulação do problema (definir variáveis, domínios, restrições)

**Passo 2)** Resolução do problema (propagar restruções e, caso necessário, usar algoritmo de procura)

Nesta aula iremos apenas dedicar-nos à formulação.

### Recursos necessários

* Para executar as experiências que se seguem, copie os seguintes ficheiros para a sua directoria de trabalho:

    * [CSP.py](https://moodle.ciencias.ulisboa.pt/pluginfile.php/79649/mod_resource/content/1/csp.py) - módulo principal 
    * [search.py](https://moodle.ciencias.ulisboa.pt/pluginfile.php/79652/mod_resource/content/1/search.py) - módulo auxiliar 
    * [utils.py](https://moodle.ciencias.ulisboa.pt/pluginfile.php/79653/mod_resource/content/1/utils.py) - módulo auxiliar

* Crie um novo módulo **labCSP.py** para ir realizando as experiências sugeridas. 

**Não altere os módulos CSP.py, search.py e utils.py**.

Comece por importar o módulo principal:

In [4]:
from csp import *

### Módulo *CSP.py* - Breve Explicação

Este módulo foi construído com base no módulo **CSP.py** disponível no [repositório habitual](https://github.com/aimacode/aima-python). 
No essencial, é disponibilizado o seguinte:
* Classe **CSP**, que serve de base à definição dos problemas CSP;
* Funções que implementam os métodos de resolução de problemas CSP, que trataremos apenas na aula seguinte.

Este módulo tem que ser importado (como ilustrado acima), mas **não deverá alterá-lo**.

**Note que este módulo lida apenas com restrições binárias!** As restrições unárias terão de ser eliminadas de modo a verificar a consistência das variáveis. Os domínio reflectem já essas restrições. 

### Criação de um CSP usando a classe **CSP**

Esta classe permite definir um CSP assumindo que os domínios das variáveis são finitos. Para definir um CSP concreto é necessário definir as variáveis, os domínios, o grafo de restrições e as restrições que serão depois usadas para criar um objecto da classe CSP.

Neste contexto, o construtor **`def __init__(self, variables, domains, neighbors, constraints)`** tem os seguintes argumentos:
  
* **`variables`** : Lista de variáveis (strings ou inteiros).

* **`domains`** : Dicionário com elementos do tipo `{var:[valor, ...]}`.

* **`neighbors`**: Dicionário com elementos do tipo `{var:[var,...]}` em que cada variável var (chave) tem como valor uma lista das variáveis com as quais tem restrições (vizinhos no grafo de restrições), que define o grafo de restrições.

* **`constraints`** : Função do tipo `f(A, a, B, b)` que devolve `True` se os vizinhos `A` e `B` satisfazem a restrição quando têm valores `A=a` e `B=b`. 

### Problema das três variáveis A, B e C tal que todas são diferentes
Vamos formular o problema de encontrar valores para as 3 variáveis A, B e C tais que sejam todas diferentes, quando o domínio é {1,2,3}
Para isso vamos definir as variáveis, os domínios, a vizinhança entre as variáveis (as que estão ligadas por restrições binárias) e a função que verifica se os valores de duas variáveis vizinhas são consistentes, não violando a respectivas restrições.

#### Definição dos domínios
Os domínios vão ser listas de números. Neste caso referem-se à lista formada por 1, 2 e 3.

O domínio vai ser o mesmo para todas as variáveis.

``` python
    dominios_ABCdifs = {}
    for v in variaveis_ABCdifs :
        dominios_ABCdifs[v] = [1,2,3]  
```

#### Definir Vizinhos
Vamos criar o grafo de restrições com os arcos seguintes:
    
    A <--> B, A <--> C, B <--> C
    
    Para isso usaremos a função 
```python
    parse_neighbors('A : B C; B: C; C : ')
```

É necessário perceber o que faz o parse_neighbors. Converte uma string da forma 'X: Y Z; Y: Z' num dicionário como este:
```python
    {'Y': ['X', 'Z'], 'X': ['Y', 'Z'], 'Z': ['X', 'Y']}
```
Declarando 'X: Y' não precisamos de declarar 'Y: X'.
A função parse_neighbors() irá devolver um defaultdict que não é mais do que um dicionário com uma função factory para gerar o valor por defeito quando se pede o valor de uma chave que ainda não existe. Neste caso, gera-se por omissão uma lista vazia. 

#### Definir a função  que verifica a satisfação das restrições entre duas variáveis afectadas
Precisamos de uma função que verifique se duas variáveis X, e Y vizinhas satisfazem todas as restrições binárias em que estão envolvidas, quando afectadas respectivamente aos valores a e b.

Essa função neste caso é a mesma para todos as variáveis vizinhas, eles têm de ter valores diferentes entre si. Vamos usar a função ***different_values_constraint()*** definida em csp.py

```python
    def different_values_constraint(A, a, B, b):
        """A constraint saying two neighboring variables must differ in value."""
        return a != b
```

Vamos então criar o CSP desejado utilizando uma função.

In [5]:
def CSP_ABCdifs():
    """
    Retorna um objecto da classe CSP inicializado com as variáveis, os domínios,
    os vizinhos e restrições do problema de em que todos são diferentes entre si.
    """
    
    #Definir Variáveis
    variaveis_ABCdifs = 'A B C'.split()
        
    # Definir Domínios
    # Devolve um dicionario com os domínios com as variáveis do problema ABCdifs.
    # Deveria ser {1,2,3} para todas as variáveis, mas dado que a implementação 
    # em CSP assume que as restrições são binárias.     
    dominios_ABCdifs = {}
    for v in variaveis_ABCdifs :
        dominios_ABCdifs[v] = [1,2,3]
    
    #Definir Vizinhos
    #Cria o grafo de restrições com os arcos seguintes:
    #A : B
    #B : C
    
    vizinhos_ABCdifs = parse_neighbors('A : B C; B: C')

        
    return CSP(variaveis_ABCdifs, dominios_ABCdifs, vizinhos_ABCdifs, different_values_constraint)

p = CSP_ABCdifs()

Podemos agora ler os atributos do objecto criado:

In [6]:
print("Variáveis = ", p.variables)
print("Domínios = ", p.domains)
print("Variáveis = ", p.neighbors)
print("Restrições",p.constraints)

Variáveis =  ['A', 'B', 'C']
Domínios =  {'A': [1, 2, 3], 'B': [1, 2, 3], 'C': [1, 2, 3]}
Variáveis =  defaultdict(<class 'list'>, {'A': ['B', 'C'], 'B': ['A', 'C'], 'C': ['A', 'B']})
Restrições <function different_values_constraint at 0x00000242B3398C10>


Invoquemos a restrição sobre duas variáveis:

In [7]:
p.constraints('A',2,'B',3)

True

In [8]:
p.constraints('A',2,'B',2)

False

### Problema dos Jogadores de Golfe (Golfers)

<img src=".\Imagens\golf.gif" alt="Drawing" style="width: 200px;"/>

Vamos agora formular o problema dos Golfers:
Quatro jogadores de golfe (Fred, Joe, Bob, Tom) estão alinhados da esquerda para a direita em posições numeradas de 1 a 4. Todos têm calças de côres diferentes. Tendo em conta a informação a seguir formule o problema de modo a determinar a ordem pela qual os golfistas estão alinhados e a côr das respectivas calças.

    Um tem calças vermelhas;
    O que está à direita do Fred tem calças azuis;
    Joe é o segundo no alinhamento;
    Bob tem calças pretas;
    Tom não está na posição 4, e não é ele que tem as calças cor de laranja.

#### Formulação do CSP Golfers

Por forma formular o problema implementamos a **função `CSP_golfers`** que devolve um objecto da classe `CSP` inicializado com as variáveis, os domínios, os vizinhos e as restrições do problema dos Golfers.

Antes de criar o objecto da classe `CSP` temos que definir:

* **`variaveis_golfers`**
* **`dominios_golfers`**
* **`vizinhos_golfers`**
* **`restricoes_golfers`**

de forma a passar estes argumentos ao construtor da classe `CSP`. 

Repare ainda que `restricoes_golfers` é uma função que avalia as restrições binárias do problema dos Golfers.

Vamos descrever a função CSP_golfers com algum detalhe, nos seus 4 componentes.

### Modelização dos Golfers
Haverão várias maneiras de modelizar os Golfers, entre elas:

    1. Podemos considerar que temos 8 variáveis: os nomes dos golfers e as cores das calças e que os domínios serão as posições emtre 1 e 4. Assim, a afectação {Bob = 1, Azuis=1} significa que o Bob está na posição 1 e quem veste calças azuis também, i.e. o Bob veste calças azuis e está na primeira posição.

    2. As variáveis serão as posições e as cores das calças e os domínios são os nomes. Nesse caso a afectação {pos_1 = Bob, azuis = Bob} quer dizer a mesma coisa.

    3. As variáveis as posições e os nomes e como domínios as cores das calças. Eis a afectação equivalente às anteriores neste modelo: {1=azuis, Bob = azuis}
    
    4. As variáveis serão as pessoas com dois atributos: cor de calças e posição. Nesse caso os domínios serão duplos de cores e de posições. A afectação equivalente às de cima será: {Bob = (azuis,1)}
    
    5. As variáveis serão as posições com dois atributos: cor de calças e nomes. Nesse caso os domínios serão duplos de cores e de nomes. A afectação equivalente às de cima será: {pos_1 = (azuis,Bob)}
    
    6. As variáveis serão as cores das calças com dois atributos: posição e nomes. Nesse caso os domínios serão duplos de posições e de nomes. A afectação equivalente às de cima será: {azuis = (1,Bob)}
    

Achamos mais simples as 3 primeiras forma de modelar este problema embora tenhamos mais variáveis os domínios sao mais simples: os números em {1,2,3,4}. Por outro lado as restrições são mais fáceis de exprimir. 

Vamos seguir o primeiro modelo.
 
    
#### Definição das variáveis
Temos 4 variáveis que se referem aos jogadores de Golf e também temos variáveis que se referem a quem tem uma determinada côr de calças.

Vamos criar duas listas com os dois conjuntos de variáveis. Podíamos ter explicitado as listas mas resolvemos usar o método split().

``` python
    golfers = 'Bob Fred Joe Tom'.split()
    cores_calcas = 'Azuis Laranja Pretas Vermelhas'.split()
```
Vamos criar uma lista com todas as variáveis:
``` python
variaveis_golfers = golfers + cores_calcas
```

#### Definição dos domínios
Os domínios vão ser listas de números. Neste caso referem-se às posições, que variam de 1 a 4.
Essas listas são guardadas num dicionário, em que as chaves são as variáveis e os valores os domínios respectivos.

O domínio deveria ser o mesmo para todas as variáveis, {1,2,3,4}, mas dado que a implementação 
em CSP assume que as restrições são binárias, as restrições unárias foram eliminadas verificando a consistência das variáveis e os domínios das variáveis 'Joe' e 'Tom' já reflectem  essas restrições.

``` python
    dominios_golfers = {}
    for v in variaveis_golfers :
        dominios_golfers[v] = [1,2,3,4]
    dominios_golfers['Joe'] = [2]
    dominios_golfers['Tom'] = [1,2,3]  
```

#### Definir Vizinhos
Vamos criar o grafo de restrições com os arcos seguintes:
    
    Bob : Fred Joe Tom Pretas
    Fred: Joe Tom Azuis
    Joe : Tom
    Tom : Laranja
    Azuis : Laranja Pretas Vermelhas
    Laranja : Pretas Vermelhas
    Pretas : Vermelhas

```python

vizinhos_golfers = parse_neighbors('Bob : Fred Joe Tom Pretas; \
        Fred: Joe Tom Azuis; Joe : Tom ; Tom : Laranja; \
        Azuis : Laranja Pretas Vermelhas; Laranja : Pretas Vermelhas; \
        Pretas : Vermelhas')
```

É necessário perceber o que faz o parse_neighbors. Converte uma string da forma 'X: Y Z; Y: Z' num dicionário como este:

``` python 
    {'Y': ['X', 'Z'], 'X': ['Y', 'Z'], 'Z': ['X', 'Y']}
```

Declarando 'X: Y' não precisamos de declarar 'Y: X'.

A função ***parse_neighbors()*** irá devolver um ***defaultdict*** que não é mais do que um dicionário com uma função **factory()** para gerar o valor por defeito quando se pede o valor de uma chave que ainda não existe. Neste caso, gera-se por omissão uma lista vazia. 

#### Definir a função  que verifica a satisfação das restrições entre duas variáveis afectadas
Precisamos de uma função que verifique se duas variáveis X, e Y satisfazem todas as restrições binárias em que estão envolvidas, quando afectadas respectivamente aos valores a e b.

Essa função vai criar todas as restrições e depois verificar a satisfabilidade de todas as que envolvam o par de variáveis. Neste caso, dos Golfers, só teremos uma restrição por par.


Cada restrição será uma função:

Eis as binárias:

        Retrições Binárias:
        R_B1) Fred = Azuis - 1  [equivalente a Azul = Fred + 1]
        R_B2) Bob = Pretas
        R_B3) Tom <> Laranja

As não binárias (globais) terão de ser transformadas em binárias envolvendo apenas o <>.

        R_G1) allDiff(golfers)
        R_G2) allDiff(cores_calcas)


Todas as condições nas restrições, serão implementadas em python como funções

``` python
def menos1(x,y):
    return (x == y - 1)
        
def mais1(x,y):
    return (x == y + 1)
        
def iguais(x,y):
    return x == y
        
def diferentes(x,y):
    return x != y

```

Vamos guardar as restrições binárias num dicionário em que cada chave é um par de variáveis e o valor é uma restrição (função). Note que vamos incluir as condições para o par (X,Y) e o seu simétrico. É necessário fazer isto porque, ao contrário da vizinhança, que é uma relação comutativa, algumas restrições, como por exemplo "menos1" não são comutativas.

```python
# restrições binárias (necessário incluir (X,Y) e (Y,X))
restricoes = { ('Fred','Azuis') : menos1, ('Azuis','Fred') : mais1, 
               ('Bob','Pretas') : iguais, ('Pretas','Bob') : iguais,
               ('Tom','Laranja') : diferentes, ('Laranja','Tom') : diferentes}
```                       
Iremos adicionar de modo automático a esse dicionário as restrições binárias que resultam das restrições globais:

```python
# all-different em cada categoria (golfers e cores)
if X in golfers and Y in golfers or X in cores_calcas and Y in cores_calcas :
    restricoes[(X,Y)] = diferentes
```
  
No final da geração das restrições, é necessário verificar se X=a e Y=b satisfazem a restrição, que depende da ordem do par; Se não houver uma restrição entre essas duas variáveis então o resultado é True.

Assim, a função restricoes_golfers(X, a, Y, b), depois de se terem definido as condições e o dicionário com as restrições, irá devolver:
        
```python
if (X,Y) in restricoes :
    return restricoes[(X,Y)]
else : # Se não há restrição ...
    return True
```



Depois da explicação em detalhe da geração dos componentes necessário para gera uma instância da classe CSP para o problemas dos Golfers, eis o código completo. 

In [9]:
# Funções binárias usadas nas restrições

def menos1(x,y):
    """x é igual a y-1"""
    return (x == y - 1)
        
def mais1(x,y):
    """x é igual a y+1"""
    return (x == y + 1)
        
def iguais(x,y):
    """x é igual a y"""
    return x == y
        
def diferentes(x,y):
    """x diferente de y"""
    return x != y


def CSP_golfers ():
    """
    Retorna um objecto da classe CSP inicializado com as variáveis, os domínios,
    os vizinhos e restrições do problema dos Golfers
    """
    
    #Definir Variáveis
    #golfers - posicao dos golfers - {1,2,3,4}
    #cores_calcas - posicao das cores - {1,2,3,4}
    golfers = 'Bob Fred Joe Tom'.split()
    cores_calcas = 'Azuis Laranja Pretas Vermelhas'.split()
    variaveis_golfers = golfers + cores_calcas
    
    # Definir Domínios
    # Devolve um dicionario com os domínios com as variáveis do problema dos Golfers.
    # Deveria ser {1,2,3,4} para todas as variáveis, mas dado que a implementação 
    # em CSP assume que as restrições são binárias, as restrições unárias foram 
    # eliminadas verificando a consistencia das variáveis e os domínios das variáveis 'Joe' e 'Tom' já reflectem 
    # essas restrições.      
    dominios_golfers = {}
    for v in variaveis_golfers :
        dominios_golfers[v] = [1,2,3,4]
    dominios_golfers['Joe'] = [2]
    dominios_golfers['Tom'] = [1,2,3]  
 
    #Definir Vizinhos
    #Cria o grafo de restrições com os arcos seguintes:
    #Bob : Fred Joe Tom Pretas
    #Fred: Joe Tom Azuis
    #Joe : Tom
    #Tom : Laranja
    #Azuis : Laranja Pretas Vermelhas
    #Laranja : Pretas Vermelhas
    #Pretas : Vermelhas
    
    vizinhos_golfers = parse_neighbors('Bob : Fred Joe Tom Pretas; \
        Fred: Joe Tom Azuis; Joe : Tom ; Tom : Laranja; \
        Azuis : Laranja Pretas Vermelhas; Laranja : Pretas Vermelhas; \
        Pretas : Vermelhas')
    

    #Definir função que verifica restrições binárias
    def restricoes_golfers(X, a, Y, b) :
        """
        A implementação em CSP assume que as restrições são binárias.
        Retorna True se (X=a,Y=b) satisfaz as restrições entre X e Y.
        
        Restrições unárias:
        R_U1) Joe = 2 obriga a que domínio Joe = {2}
        R_U2) Tom <> 4 obriga a que domínio Tom = {1,2,3}
        
        Retrições Binárias:
        R_B1) Fred = Azuis - 1  [equivalente a Azul = Fred + 1]
        R_B2) Bob = Pretas
        R_B3) Tom <> Laranja
        
        Restrições Globais:
        R_G1) allDiff(golfers)
        R_G2) allDiff(cores_calcas)
        """
        
        # condições envolvidas nas restrições binárias
        

        # restrições binárias (necessário incluir (X,Y) e (Y,X))
        restricoes = { ('Fred','Azuis') : menos1, ('Azuis','Fred') : mais1, 
                       ('Bob','Pretas') : iguais, ('Pretas','Bob') : iguais,
                       ('Tom','Laranja') : diferentes, ('Laranja','Tom') : diferentes}

        
        # all-different em cada categoria (golfers e cores)
        if X in golfers and Y in golfers or \
           X in cores_calcas and Y in cores_calcas :
            restricoes[(X,Y)] = diferentes 
        
        if (X,Y) in restricoes :
            return restricoes[(X,Y)](a,b)

        
    return CSP(variaveis_golfers, dominios_golfers, vizinhos_golfers, restricoes_golfers)

Vamos agora criar um objecto `CSP` usando a função `CSP_golfers` que implementámos em cima:

In [10]:
p = CSP_golfers()

E verificar os valores das variáveis, dos domínios e dos vizinhos (grafo de restrições):

In [11]:
print("Variáveis = ", p.variables)

Variáveis =  ['Bob', 'Fred', 'Joe', 'Tom', 'Azuis', 'Laranja', 'Pretas', 'Vermelhas']


In [12]:
print("Domínios = ", p.domains)

Domínios =  {'Bob': [1, 2, 3, 4], 'Fred': [1, 2, 3, 4], 'Joe': [2], 'Tom': [1, 2, 3], 'Azuis': [1, 2, 3, 4], 'Laranja': [1, 2, 3, 4], 'Pretas': [1, 2, 3, 4], 'Vermelhas': [1, 2, 3, 4]}


In [14]:
print("Vizinhos = ", p.neighbors)

Vizinhos =  defaultdict(<class 'list'>, {'Bob': ['Fred', 'Joe', 'Tom', 'Pretas'], 'Fred': ['Bob', 'Joe', 'Tom', 'Azuis'], 'Joe': ['Bob', 'Fred', 'Tom'], 'Tom': ['Bob', 'Fred', 'Joe', 'Laranja'], 'Pretas': ['Bob', 'Azuis', 'Laranja', 'Vermelhas'], 'Azuis': ['Fred', 'Laranja', 'Pretas', 'Vermelhas'], 'Laranja': ['Tom', 'Azuis', 'Pretas', 'Vermelhas'], 'Vermelhas': ['Azuis', 'Laranja', 'Pretas']})


Verifique também que a função `restricoes_golfers` foi bem implementada, testando algumas afectações a pares de variáveis, confirmando que a função retorna `True`, se os valores não têm conflito com as restrições entre o par de variáveis passados como argumento para o par de valores concreto, e `False`, caso contrário.

In [15]:
# Restrição Fred = Azuis - 1
print(p.constraints('Fred', 2, "Azuis", 3))
print(p.constraints('Fred', 3, "Azuis", 2))

True
False


In [16]:
#Restrição Tom <> Laranja
print(p.constraints('Tom', 1, "Laranja", 1))
print(p.constraints('Tom', 1, "Azuis", 2))

False
None


In [17]:
#Restrição alldiff golfers
print(p.constraints('Bob', 1, "Fred", 2))
print(p.constraints('Bob', 1, "Fred", 1))   

True
False


In [18]:
#Restrição alldiff cores calças
print(p.constraints('Azuis', 1, "Laranja", 1))
print(p.constraints('Azuis', 1, "Laranja", 2))

False
True


### Exercício 1
Formule usando a metodologia CSP, o problema de encontrar valores para as 3 variáveis A, B e C tais que A > B > C em que o domínio é {1,2,3} para todas as variáveis.

In [4]:
## Solução do exercício 1

def CSP_ABCdifs():
    
    #Definir Variáveis
    variaveis_ABCdifs = ['A','B','C','D','E']
        
    # Definir Domínios
    # Devolve um dicionario com os domínios com as variáveis do problema ABCdifs.
    # Deveria ser {1,2,3} para todas as variáveis, mas dado que a implementação 
    # em CSP assume que as restrições são binárias.     
    dominios_ABCdifs = {}
    for v in variaveis_ABCdifs :
        dominios_ABCdifs[v] = [0,1,2,3,4]
    dominios_ABCdifs['C'] != [0]
    dominios_ABCdifs['E'] != [4]
    dominios_ABCdifs['A'] >= dominios_ABCdifs['C']
    dominios_ABCdifs['B'] > dominios_ABCdifs['E']
    dominios_ABCdifs['B'] > dominios_ABCdifs['D']
    dominios_ABCdifs['B'] = dominios_ABCdifs['C']
    
    #Definir Vizinhos
    #Cria o grafo de restrições com os arcos seguintes:
    #A : B
    #B : C
    
    vizinhos_ABCdifs = parse_neighbors('A : B C; B: C')

        
    return CSP(variaveis_ABCdifs, dominios_ABCdifs, vizinhos_ABCdifs, different_values_constraint)


p = CSP_ABCdifs()
print("Domínios = ", p.domains)

NameError: name 'vizinhos_ABCdifs' is not defined

### Exercício 2 - Problema da coloração de mapas (Map Coloring)
<img src="./Imagens/mapa-colorir.PNG
          " alt="Drawing" style="width: 200px;"/>
Considere o problema de colorir o mapa em cima usando 4 cores. Defina a função **`CSP_map_coloring`** que devolve um `CSP` inicializado com as variáveis, os domínios, os vizinhos e as restrições para este problema.

### Exercício 3 - Problema genérico de coloração de mapa
Crie uma função que permite criar qualquer problema de coloração de mapas, fornecendo-lhe como input o conjunto de cores e a vizinhança. As cores definem o domínio de todas as variáveis e a partir da vizinhança obtêm-se as variáveis. Teste-o com o mapa do exercício anterior e com os seguintes países, dando-lhes as regiões adjacentes e as cores.

```python
australia = MapColoringCSP(list('RGBY'),
                           'SA: WA NT Q NSW V; NT: WA Q; NSW: Q V; T: ')

france = MapColoringCSP(list('RGBY'),
                        """AL: LO FC; AQ: MP LI PC; AU: LI CE BO RA LR MP; BO: CE IF CA FC RA
        AU; BR: NB PL; CA: IF PI LO FC BO; CE: PL NB NH IF BO AU LI PC; FC: BO
        CA LO AL RA; IF: NH PI CA BO CE; LI: PC CE AU MP AQ; LO: CA AL FC; LR:
        MP AU RA PA; MP: AQ LI AU LR; NB: NH CE PL BR; NH: PI IF CE NB; NO:
        PI; PA: LR RA; PC: PL CE LI AQ; PI: NH NO CA IF; PL: BR NB CE PC; RA:
        AU BO FC PA LR""")

usa = MapColoringCSP(list('RGBY'),
                     """WA: OR ID; OR: ID NV CA; CA: NV AZ; NV: ID UT AZ; ID: MT WY UT;
        UT: WY CO AZ; MT: ND SD WY; WY: SD NE CO; CO: NE KA OK NM; NM: OK TX AZ;
        ND: MN SD; SD: MN IA NE; NE: IA MO KA; KA: MO OK; OK: MO AR TX;
        TX: AR LA; MN: WI IA; IA: WI IL MO; MO: IL KY TN AR; AR: MS TN LA;
        LA: MS; WI: MI IL; IL: IN KY; IN: OH KY; MS: TN AL; AL: TN GA FL;
        MI: OH IN; OH: PA WV KY; KY: WV VA TN; TN: VA NC GA; GA: NC SC FL;
        PA: NY NJ DE MD WV; WV: MD VA; VA: MD DC NC; NC: SC; NY: VT MA CT NJ;
        NJ: DE; DE: MD; MD: DC; VT: NH MA; MA: NH RI CT; CT: RI; ME: NH;
        HI: ; AK: """)

```

### Exercício 4 - Palavras Cruzadas pequeninas
<img src=".\Imagens\PalavrasCruzadas2x2.png" alt="Drawing" style="width: 200px;"/>
Modelize um problema de palavras cruzadas usando CSP, em que temos uma grelha 2x2 e 8 palavras possíveis para dispor 4 delas nas 2 linhas e 2 colunas. Cada palavra só pode aparecer numa das 4 regiões possíveis (2 linhas e 2 colunas) e as letras das palavras têm de ser as mesmas quando as palavras se intersectam. Eis uma solução para o problema com 8 palavras: {MA, OS, AS, TU, LI, RO, TO, MO}

### Exercício 5 - Escalonamento de tarefas
<img src=".\Imagens\tarefas.PNG" alt="Drawing" style="width: 200px;"/>
Formule o problema de escalonamento de tarefas de modo a determinar os tempos de início de cada tarefa que minimizam a duração total do processo.

### Exercício 6 - Problema das 4 rainhas

<img src=".\Imagens\4queens.png" alt="Drawing" style="width: 200px;"/>
Considere o problema das 4 rainhas em que queremos dispor 4 raínhas num tabuleiro 4x4 tal que as raínhas não se ataquem entre si. As raínhas atacam ao longo das colunas, linhas e diagonais. Defina a função **`CSP_4_rainhas`** que devolve um CSP inicializado com as variáveis, os domínios, os vizinhos e as restrições para este problema.

### Exercício 7 - Problema das N rainhas
<img src=".\Imagens\16-queens-problem-trial-1-4-638.jpg" alt="Drawing" style="width: 200px;"/>

Altere o problema das 4 rainhas que resolveu no exercício anterior e formule o problema das `N` rainhas. Defina a função **`CSP_N_rainhas(N)`** que recebe como parâmetro o número `N` de rainhas e devolve um `CSP` inicializado com as variáveis, os domínios, os vizinhos e as restrições para este problema.

### Exercício 8 - Sudoku

Modelize como um CSP o problema do Sudoku para uma grelha 9x9. Eis o exemplo de um problema em modo texto que pode usar para construir o CSP. Construa uma função que recebe a string com o exemplo de puzzle e devolve a instância de CSP respectiva. 

```python
..3.2.6..
9..3.5..1
..18.64..
..81.29..
7.......8
..67.82..
..26.95..
8..2.3..9
..5.1.3..
```

temos a string de input:

'..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'

### Exercício 9 - Puzzle da Zebra
Considere o puzzle da Zebra [https://en.wikipedia.org/wiki/Zebra_Puzzle] Considere o seguinte puzzle lógico: Em 5 casas, cada uma com uma cor diferente, vivem 5 pessoas de nacionalidades diferentes, cada um prefere uma marca diferente de chocolates, uma bebida diferente, e um animal diferente. Dados os factos que se seguem, as perguntas a responder são: “Onde mora a zebra e em que casa bebem água “. Formule o problema usando CSP.

<img src=".\Imagens\zebra.PNG" alt="Drawing" style="width: 600px;"/>